# Web Scrapping Comparis

In [1]:
import os
import math
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from getpass import getpass
import csv
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# 1. Get the links for all flats announced in City X

In [2]:
# select city and define url
city = 'Bern'.lower()
url = "https://www.comparis.ch/immobilien/marktplatz/"+city+"/wohnung/mieten?page=0"
url

'https://www.comparis.ch/immobilien/marktplatz/bern/wohnung/mieten?page=0'

In [37]:
wdpath = "chromedriver"
driver = webdriver.Chrome(wdpath)

## 1.1 Webscrapping for page 1

In [38]:
url="https://www.comparis.ch/immobilien/marktplatz/bern/wohnung/mieten?page=2"
driver.get(url)

In [39]:
## minimize the size of the side, so that we get all the links
driver.execute_script("document.body.style.zoom = '0.1'")

In [40]:
flats_link = driver.find_elements_by_class_name("excbu0j3")

In [41]:
flat_links = list(set([flats.get_attribute('href') for flats in flats_link]))
flat_links

['',
 'https://www.comparis.ch/immobilien/marktplatz/bern/wohnung/3-zimmer/mieten',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23016600',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014342',
 'https://www.instagram.com/comparis_ch/',
 'https://www.comparis.ch/immobilien/result/favoritelist',
 'https://www.comparis.ch/immobilien/marktplatz/bern/studio/mieten',
 'https://www.comparis.ch/immobilien/marktplatz/bern/moebilierte-wohnung/mieten',
 'https://it.comparis.ch/immobilien/marktplatz/bern/wohnung/mieten?page=2',
 'https://www.comparis.ch/comparis/info/legal',
 'https://www.linkedin.com/company/comparis.ch-ag',
 'https://fr.comparis.ch/immobilien/marktplatz/bern/wohnung/mieten?page=2',
 'https://www.comparis.ch/immobilien/marktplatz/bern/weissenbuehl/wohnung/mieten',
 'https://www.comparis.ch/comparis/newsletter/subscribe',
 'https://www.comparis.ch/immobilien/marktplatz/bern/wohnung/mieten?page=2',
 'https://www.facebook.com/comparis.ch',
 'https:

In [42]:
links = list(set([x if len(str(re.findall('https://www.comparis.ch/immobilien/marktplatz/details/show/', str(x))))>=3 else 0 for x in flat_links]))[1:]

links

['https://www.comparis.ch/immobilien/marktplatz/details/show/23016581',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23016600',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014342',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23017350',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23016559',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014346',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23016606',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014344',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014339',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23018295']

In [43]:
print(len(links))

10


## 1.2. Get number of result pages

In [44]:
#amount of links
nr = list(set([x[-3:] if len(str(re.findall('page', str(x))))>=3 else 0 for x in flat_links]))[0:]


nrlis = ["".join(re.findall('[0-9]', str(x))) for x in nr]
resultpages=int(max(nrlis))
resultpages


78

## 1.3 Get links from flats from all the result pages

In [45]:
urlOhnePage = url[:-1]
for i in range(1, (resultpages-1)):
    page_url = urlOhnePage + str(i)
    driver.get(page_url)
    driver.execute_script("document.body.style.zoom = '0.1'")
    time.sleep(2.342)
    flats_link = driver.find_elements_by_class_name("excbu0j3")
    flat_links = list(set([flats.get_attribute('href') for flats in flats_link]))
    lin= list(set([x if len(str(re.findall('https://www.comparis.ch/immobilien/marktplatz/details/show/', str(x))))>=3 else 0 for x in flat_links]))[1:]
    for x in lin: 
        links.append(x)

## 1.4 Save lis of links to a csv

In [46]:
len(links)

722

In [47]:
linklist = pd.DataFrame(links, columns =['link'])
linklist.to_csv('links_'+city+'.csv')

# 2. Detailed Infos for each flat 

## 2.1 Load Links and change type to list

In [48]:
linklist = pd.read_csv('links_'+city+'.csv',  index_col=0)

In [49]:
linklist=list(linklist['link'])

In [51]:
len(linklist)

722

## 2.2 Infos for one flat

#### 2.2.0 get soup

In [52]:
url2 = linklist[0]

html2 = requests.get(url2).content
html2

soup2 = BeautifulSoup(html2, "lxml")  # Check documentation to specify 'features'
soup2

<!DOCTYPE html>
<html lang="De">
<head>
<title>3 Zimmer Wohnung in 3007 Bern  – Immobilienmarkt von comparis.ch</title>
<meta content="follow,noindex" name="robots"/>
<meta content="Wohnung, 3 Zimmer Wohnung, Miete, 3007 Bern " name="keywords"/>
<meta content="3 Zimmer Wohnung in 3007 Bern , Mietpreis / Monat 1480 CHF  , Balkon, Lift" name="description"/>
<meta content="comparis.ch AG" name="copyright"/>
<meta content="de" http-equiv="Content-Language"/>
<meta content="de" name="language"/>
<meta content="private" http-equiv="Cache-Control"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"30d2eb686c","applicationID":"63661895","transactionName":"NAYHNkEFVhUCVUQICw1MJDFjS20PKlhEBAMRAhELXApwBw1SXAQW","queueTime":1,"applicationTime":535,"agent":"","atts":""}</script><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpi

In [53]:
nr = int(url2[-8:])
nr

23016581

In [54]:
source =pd.DataFrame.from_dict({'city': city, 'DateofScrapping': '26.02.2020'}, orient='index', columns = [nr] )
source

,23016581
city,bern
DateofScrapping,26.02.2020


#### 2.2.1  Adress and Date of Publication

In [55]:
###

"""
<h3 class="text-green">Riedbachstrasse 80, 3027 Bern</h3>

"""
adress_soup = soup2.find_all('h3', attrs = {'class':'text-green'})
if len(adress_soup)==0:
    adress = {'adress': 'NaN'}
else:
    adress = {'adress': adress_soup[0].text}
adress
adress = pd.DataFrame.from_dict(adress, orient='index', columns = [nr] )
adress

,23016581
adress,"Konsumstrasse 8, 3007 Bern"


In [56]:
#Date of Publication

"""
<div class="row xsmall-12 columns">
        <ul class="meta-tag-line">
            <li>
                Erstmals gefunden am 26.02.2020
            </li>
        </ul>
    </div>
"""

publication={}
publication_soup = soup2.find_all('div', attrs = {'class':'row xsmall-12 columns'})
for element in publication_soup:
    for x in element.find_all('li'):
        publication['publication_date']=(x.text.replace('\r\n','').replace('Erstmals gefunden am ','').replace('\r\n','').replace(" ",''))
publication
publication_date = pd.DataFrame.from_dict(publication, orient='index', columns = [nr] )
publication_date

,23016581
publication_date,25.02.2020


#### (##2.2.2  Overview (First large table)##)

In [57]:
## Overview from one flat --> not used?
overview=[]
overview_soup = soup2.find_all('div', attrs = {'class':'item-price large'})
for element in overview_soup:
    for x in element.find_all('strong'):
        overview.append(x.text)
overview


['Wohnung', "CHF 1'480", '3', '68 m²', 'ca. CHF 187 /Jahr']

#### 2.2.3 Ratings

In [58]:
#### rating
"""
<div class="info-grid-content">
                    <span class="item-rating medium">
                        <strong>
                            <i class="text-green icon-comparis-points" data-icon="data-icon">
</i> 4
                        </strong>
                    </span>
                </div>
"""
rating_soup = soup2.find_all('span', attrs = {'class':'item-rating medium'})
if len(rating_soup)==0:
    rating = {'rating': 'NaN'}
else:
    rating = {'rating': re.findall('[0-9]',rating_soup[0].text)[0]}
rating
rating = pd.DataFrame.from_dict(rating, orient='index', columns = [nr] )
rating

,23016581
rating,NaN


In [59]:
rating_soup

[]

#### 2.2.4 Detailed overview (second smaller table)

In [60]:
infos_soup = soup2.find_all('dl')

### Categorie name
info1=[]
for x in infos_soup[0].find_all('dt'):
    info1.append(x.text.replace(" ","").replace("\r\n",""))

info1

### actual info

info2=[]
for x in infos_soup[0].find_all('dd'):
    info2.append(x.text.replace('\r\n','').replace('CHF','').replace(' ','').replace('\n\n', '1').replace("'","").replace("m²","").replace('½','.5'))

info2

info={}
for x in range(len(info2)):
    info[info1[x]] = info2[x]

info = pd.DataFrame.from_dict(info, orient='index', columns = [nr] )
info


,23016581
Objektart,Wohnung
MietpreisproMonat,1480
MietpreisproMonat(exkl.NK),1240
Nebenkosten,240
Wohnfläche,68
Zimmer,3
Geschoss,2.Stock
Verfügbarab,01.05.2020
Balkon(e),1
Lift,1


##### 2.6 to one column

In [61]:
ds=pd.concat([info, rating, adress, publication_date, source])

In [62]:
ds = ds.transpose()
ds

,Objektart,MietpreisproMonat,MietpreisproMonat(exkl.NK),Nebenkosten,Wohnfläche,Zimmer,Geschoss,Verfügbarab,Balkon(e),Lift,AltesGebäude,ÖV-Haltestelle,Einkaufsmöglichkeit,rating,adress,publication_date,city,DateofScrapping
23016581,Wohnung,1480,1240,240,68,3,2.Stock,01.05.2020,1,1,1,40m,50m,NaN,"Konsumstrasse 8, 3007 Bern",25.02.2020,bern,26.02.2020


# 3. Repeat for other flats

In [63]:
## repeat in a loop with all url
for x in linklist:
    print(x)


https://www.comparis.ch/immobilien/marktplatz/details/show/23016581
https://www.comparis.ch/immobilien/marktplatz/details/show/23016600
https://www.comparis.ch/immobilien/marktplatz/details/show/23014342
https://www.comparis.ch/immobilien/marktplatz/details/show/23017350
https://www.comparis.ch/immobilien/marktplatz/details/show/23016559
https://www.comparis.ch/immobilien/marktplatz/details/show/23014346
https://www.comparis.ch/immobilien/marktplatz/details/show/23016606
https://www.comparis.ch/immobilien/marktplatz/details/show/23014344
https://www.comparis.ch/immobilien/marktplatz/details/show/23014339
https://www.comparis.ch/immobilien/marktplatz/details/show/23018295
https://www.comparis.ch/immobilien/marktplatz/details/show/23023602
https://www.comparis.ch/immobilien/marktplatz/details/show/23023377
https://www.comparis.ch/immobilien/marktplatz/details/show/23023371
https://www.comparis.ch/immobilien/marktplatz/details/show/23020985
https://www.comparis.ch/immobilien/marktplatz/de

In [64]:
link=linklist[1:]
link

['https://www.comparis.ch/immobilien/marktplatz/details/show/23016600',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014342',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23017350',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23016559',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014346',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23016606',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014344',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23014339',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23018295',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23023602',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23023377',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23023371',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/23020985',
 'https://www.comparis.ch/immobilien/marktplatz/details/show/230

In [65]:
## repeat in a loop with all url
for x in link:
    print(x)
    # get soup (2.1):
    html2 = requests.get(x).content
    soup2 = BeautifulSoup(html2, "lxml")
    nr = int(x[-8:])   
    # get id:
    nr = int(x[-8:])
    ## city and date of webscrapping
    source =pd.DataFrame.from_dict({'city': city, 'DateofScrapping': '26.02.2020'}, orient='index', columns = [nr] )
    
    # adress:
    adress_soup = soup2.find_all('h3', attrs = {'class':'text-green'})
    if len(adress_soup)==0:
        adress = {'adress': 'NaN'}
    else:
        adress = {'adress': adress_soup[0].text}
    adress = pd.DataFrame.from_dict(adress, orient='index', columns = [nr] )

    #publication Date:
    publication={}
    publication_soup = soup2.find_all('div', attrs = {'class':'row xsmall-12 columns'})
    for element in publication_soup:
        for x in element.find_all('li'):
            publication['publication_date']=(x.text.replace('\r\n','').replace('Erstmals gefunden am ','').replace('\r\n','').replace(" ",''))
    publication_date = pd.DataFrame.from_dict(publication, orient='index', columns = [nr] )
    
    
    # rating: 
    rating_soup = soup2.find_all('span', attrs = {'class':'item-rating medium'})
    if len(rating_soup)==0:
        rating = {'rating': 'NaN'}
    else:
        rating = {'rating': re.findall('[0-9]',rating_soup[0].text)[0]}
    rating = pd.DataFrame.from_dict(rating, orient='index', columns = [nr] )

    
    # details: 
    infos_soup = soup2.find_all('dl')
    # Categorie name
    info1=[]
    for x in infos_soup[0].find_all('dt'):
        info1.append(x.text.replace(" ","").replace("\r\n",""))
    # actual info
    info2=[]
    for x in infos_soup[0].find_all('dd'):
        info2.append(x.text.replace('\r\n','').replace('CHF','').replace(' ','').replace('\n\n', '1').replace("'","").replace("m²","").replace('½','.5'))
    info={}
    for x in range(len(info2)):
        info[info1[x]] = info2[x]
    info = pd.DataFrame.from_dict(info, orient='index', columns = [nr] )
    
    #row
    row =pd.concat([info, rating, adress, publication_date, source])
    row = row.transpose()
    # merge with dataset
    ds=pd.concat([ds, row], sort=False)

https://www.comparis.ch/immobilien/marktplatz/details/show/23016600
https://www.comparis.ch/immobilien/marktplatz/details/show/23014342
https://www.comparis.ch/immobilien/marktplatz/details/show/23017350
https://www.comparis.ch/immobilien/marktplatz/details/show/23016559
https://www.comparis.ch/immobilien/marktplatz/details/show/23014346
https://www.comparis.ch/immobilien/marktplatz/details/show/23016606
https://www.comparis.ch/immobilien/marktplatz/details/show/23014344
https://www.comparis.ch/immobilien/marktplatz/details/show/23014339
https://www.comparis.ch/immobilien/marktplatz/details/show/23018295
https://www.comparis.ch/immobilien/marktplatz/details/show/23023602
https://www.comparis.ch/immobilien/marktplatz/details/show/23023377
https://www.comparis.ch/immobilien/marktplatz/details/show/23023371
https://www.comparis.ch/immobilien/marktplatz/details/show/23020985
https://www.comparis.ch/immobilien/marktplatz/details/show/23020983
https://www.comparis.ch/immobilien/marktplatz/de

https://www.comparis.ch/immobilien/marktplatz/details/show/22988431
https://www.comparis.ch/immobilien/marktplatz/details/show/22985629
https://www.comparis.ch/immobilien/marktplatz/details/show/22985339
https://www.comparis.ch/immobilien/marktplatz/details/show/22985624
https://www.comparis.ch/immobilien/marktplatz/details/show/22983541
https://www.comparis.ch/immobilien/marktplatz/details/show/22984174
https://www.comparis.ch/immobilien/marktplatz/details/show/22985498
https://www.comparis.ch/immobilien/marktplatz/details/show/22984975
https://www.comparis.ch/immobilien/marktplatz/details/show/22988283
https://www.comparis.ch/immobilien/marktplatz/details/show/22982410
https://www.comparis.ch/immobilien/marktplatz/details/show/22981792
https://www.comparis.ch/immobilien/marktplatz/details/show/22978925
https://www.comparis.ch/immobilien/marktplatz/details/show/22983111
https://www.comparis.ch/immobilien/marktplatz/details/show/22978923
https://www.comparis.ch/immobilien/marktplatz/de

https://www.comparis.ch/immobilien/marktplatz/details/show/22942586
https://www.comparis.ch/immobilien/marktplatz/details/show/22940018
https://www.comparis.ch/immobilien/marktplatz/details/show/22943190
https://www.comparis.ch/immobilien/marktplatz/details/show/22943189
https://www.comparis.ch/immobilien/marktplatz/details/show/22939433
https://www.comparis.ch/immobilien/marktplatz/details/show/22939382
https://www.comparis.ch/immobilien/marktplatz/details/show/22939407
https://www.comparis.ch/immobilien/marktplatz/details/show/22939305
https://www.comparis.ch/immobilien/marktplatz/details/show/22939638
https://www.comparis.ch/immobilien/marktplatz/details/show/22939332
https://www.comparis.ch/immobilien/marktplatz/details/show/22999552
https://www.comparis.ch/immobilien/marktplatz/details/show/22939276
https://www.comparis.ch/immobilien/marktplatz/details/show/22939313
https://www.comparis.ch/immobilien/marktplatz/details/show/22938234
https://www.comparis.ch/immobilien/marktplatz/de

https://www.comparis.ch/immobilien/marktplatz/details/show/22887292
https://www.comparis.ch/immobilien/marktplatz/details/show/22899979
https://www.comparis.ch/immobilien/marktplatz/details/show/22882344
https://www.comparis.ch/immobilien/marktplatz/details/show/22882504
https://www.comparis.ch/immobilien/marktplatz/details/show/22882630
https://www.comparis.ch/immobilien/marktplatz/details/show/23014341
https://www.comparis.ch/immobilien/marktplatz/details/show/22874742
https://www.comparis.ch/immobilien/marktplatz/details/show/22881488
https://www.comparis.ch/immobilien/marktplatz/details/show/22879005
https://www.comparis.ch/immobilien/marktplatz/details/show/13963125
https://www.comparis.ch/immobilien/marktplatz/details/show/22880969
https://www.comparis.ch/immobilien/marktplatz/details/show/22880464
https://www.comparis.ch/immobilien/marktplatz/details/show/22998672
https://www.comparis.ch/immobilien/marktplatz/details/show/22879392
https://www.comparis.ch/immobilien/marktplatz/de

https://www.comparis.ch/immobilien/marktplatz/details/show/22749729
https://www.comparis.ch/immobilien/marktplatz/details/show/13443727
https://www.comparis.ch/immobilien/marktplatz/details/show/22745340
https://www.comparis.ch/immobilien/marktplatz/details/show/22751367
https://www.comparis.ch/immobilien/marktplatz/details/show/22745150
https://www.comparis.ch/immobilien/marktplatz/details/show/22748294
https://www.comparis.ch/immobilien/marktplatz/details/show/22992348
https://www.comparis.ch/immobilien/marktplatz/details/show/22869511
https://www.comparis.ch/immobilien/marktplatz/details/show/22737231
https://www.comparis.ch/immobilien/marktplatz/details/show/22739903
https://www.comparis.ch/immobilien/marktplatz/details/show/22742331
https://www.comparis.ch/immobilien/marktplatz/details/show/22742980
https://www.comparis.ch/immobilien/marktplatz/details/show/22979612
https://www.comparis.ch/immobilien/marktplatz/details/show/22780636
https://www.comparis.ch/immobilien/marktplatz/de

https://www.comparis.ch/immobilien/marktplatz/details/show/22879575
https://www.comparis.ch/immobilien/marktplatz/details/show/22535394
https://www.comparis.ch/immobilien/marktplatz/details/show/22539051
https://www.comparis.ch/immobilien/marktplatz/details/show/22519198
https://www.comparis.ch/immobilien/marktplatz/details/show/23018297
https://www.comparis.ch/immobilien/marktplatz/details/show/22881998
https://www.comparis.ch/immobilien/marktplatz/details/show/22505369
https://www.comparis.ch/immobilien/marktplatz/details/show/22562773
https://www.comparis.ch/immobilien/marktplatz/details/show/22919654
https://www.comparis.ch/immobilien/marktplatz/details/show/22121564
https://www.comparis.ch/immobilien/marktplatz/details/show/22852017
https://www.comparis.ch/immobilien/marktplatz/details/show/22500114
https://www.comparis.ch/immobilien/marktplatz/details/show/22513324
https://www.comparis.ch/immobilien/marktplatz/details/show/23024695
https://www.comparis.ch/immobilien/marktplatz/de

In [66]:
ds

,Objektart,MietpreisproMonat,MietpreisproMonat(exkl.NK),Nebenkosten,Wohnfläche,Zimmer,Geschoss,Verfügbarab,Balkon(e),Lift,...,Kubatur,Hochparterre,Neubau,Wohnungen,Wasseranschluss,Toiletten,Mittelhaus,Möbliert,JährlicherMietpreisprom²,JährlicherMietpreisprom²(exkl.NK)
23016581,Wohnung,1480,1240,240,68,3,2.Stock,01.05.2020,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23016600,Wohnung,3500,3200,300,140,5,2.Stock,01.05.2020,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23014342,Wohnung,1710,1470,240,73,3.5,NaN,01.03.2020,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23017350,Wohnung,1180,1030,150,NaN,2,1.Stock,01.05.2020,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23016559,Wohnung,1360,1230,130,44,2.5,3.Stock,01.06.2020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21480203,Wohnung,1804,1524,280,90,3,3.Stock,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21056128,Wohnung,1150,NaN,NaN,NaN,1,2.Stock,nachVereinbarung,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21202121,Wohnung,1790,1510,280,89,3,3.Stock,sofort,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21161839,Loft,2800,2500,300,144,1,EG,sofort,1,1,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3.3 Save to csv

In [67]:
ds.to_csv('flats_'+ city + '.csv')

In [68]:
ds.columns

Index(['Objektart', 'MietpreisproMonat', 'MietpreisproMonat(exkl.NK)',
       'Nebenkosten', 'Wohnfläche', 'Zimmer', 'Geschoss', 'Verfügbarab',
       'Balkon(e)', 'Lift', 'AltesGebäude', 'ÖV-Haltestelle',
       'Einkaufsmöglichkeit', 'rating', 'adress', 'publication_date', 'city',
       'DateofScrapping', 'Nutzfläche', 'Baujahr', 'Aussenparkplatz',
       'Haustiereerlaubt', 'Kabel-TV', 'Kinderfreundlich', 'Kindergarten',
       'Primarschule', 'Sekundarschule', 'Autobahn', 'Einstellplatz',
       'Rollstuhlgängig', 'Minergie-zertifiziert', 'Cheminée', 'Kubatur',
       'Hochparterre', 'Neubau', 'Wohnungen', 'Wasseranschluss', 'Toiletten',
       'Mittelhaus', 'Möbliert', 'JährlicherMietpreisprom²',
       'JährlicherMietpreisprom²(exkl.NK)'],
      dtype='object')